# 3. Sharpness-aware minimization (SAM) wirh Sparse Networks - 25 points

## 3.1 Get a sparse networks through pruning
**Pruning** is a technique used to reduce the size and complexity of a neural network model by removing (setting to zero) less important parameters. The goal is to create a more efficient model that retains its predictive accuracy while being smaller, which can improve both inference speed and memory usage.

Let's train a simple model on the MNIST dataset to learn about pruning at first. We just use 10% of the dataset for both training and testing.

### 3.1.1 Train a dense network with SGD
Let us first train a dense model with SGD. We reuse the model for the discriminator of the GAN in Homework 2 and name it 'Classifier'.

In [8]:
from lib.part3.utils import *
max_epochs = 10
#device = "cpu"  # Change this if you can and want to use a GPU device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Training on {device}")
model = Classifier().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)

Training on cuda


We define the optimizing process of SGD.

In [9]:
def optimize_sgd(model, optimizer, img, label):
    optimizer.zero_grad()
    output = model(img)
    loss = cross_entropy(output, label)
    loss.backward()
    optimizer.step()

The following cell runs the training loop, this might take a few minutes.

In [10]:
train_model(model, optimizer, optimize_sgd, max_epochs)

Epoch 0 with 0.933 accuracy on the validation set.
Epoch 1 with 0.948 accuracy on the validation set.
Epoch 2 with 0.956 accuracy on the validation set.
Epoch 3 with 0.957 accuracy on the validation set.
Epoch 4 with 0.962 accuracy on the validation set.
Epoch 5 with 0.963 accuracy on the validation set.
Epoch 6 with 0.967 accuracy on the validation set.
Epoch 7 with 0.968 accuracy on the validation set.
Epoch 8 with 0.969 accuracy on the validation set.
Epoch 9 with 0.97 accuracy on the validation set.


#### Evaluate model
Evaluate the model on the test set.

In [11]:
acc = evaluate(model)
print(f"Accuracy of {round(acc, 4)} on the test set.")

Accuracy of 0.9771 on the test set.


### 3.1.2 Sparse network with magnitude-based pruning

Magnitude-based pruning specifically focuses on **removing weights that have the smallest absolute values**, under the assumption that weights with smaller magnitudes contribute less to the model's output.

**(1)** (6 points) Realize magnitude-based pruning below, which removes a part of weights that have the smallest absolute values.

In [12]:
def magnitude_prune(model, prune_fraction):
    for name, param in model.named_parameters():
        if "weight" in name and param.requires_grad:
            # FILL: Get weight's absolute values
            weight_abs = param.data.abs()
            # FILL: Compute the threshold
            threshold = torch.quantile(weight_abs, prune_fraction)
            # FILL: Prune weights below the threshold
            param.data[weight_abs < threshold] = 0
    return model

In [13]:
import copy
# Copy a model for pruning
sparse_model = copy.deepcopy(model)
# Get a sparse model by pruning 50% parameters
sparse_model = magnitude_prune(sparse_model, prune_fraction=0.5)

Copy a sparse model for SAM implementation later in 3.2.

In [14]:
sparse_model_sam = copy.deepcopy(sparse_model)

Evaluation after pruning

In [15]:
acc = evaluate(sparse_model)
print(f"Accuracy of {round(acc, 4)} on the test set.")

Accuracy of 0.8674 on the test set.


### 3.1.3 Finetune the sparse model

We finetune the sparse model after pruning with SGD to recover its performance.

In [16]:
finetune_epoch = 3
train_model(sparse_model, optimizer, optimize_sgd, finetune_epoch)

Epoch 0 with 0.918 accuracy on the validation set.
Epoch 1 with 0.918 accuracy on the validation set.
Epoch 2 with 0.918 accuracy on the validation set.


Evaluate the sparse model after finetuning.

In [17]:
acc = evaluate(sparse_model)
print(f"Accuracy of {round(acc, 4)} on the test set.")

Accuracy of 0.929 on the test set.


**(2)** (2 point) What are the pros and cons of sparse networks?

<font color=‘blue’>

**Pros:** 
1. Reduces computational cost and memory usage, zero weights do not contribute in matrix multiplication and are not stored.
2. Sparse networks act as regularizer, improve generalization and reduce overfitting.

**Cons:**
1. Implementation is more complex, requires additional steps for pruning and finetuning.
2. Pruning can lead to loss of information and degrade performance if not done properly.
3. Modern hardware is optimized for dense matrix multiplication, sparse networks may not be as efficient.


## 3.2 Train the sparse model with SAM

Sharpness-aware minimization (SAM) is a new optimization technique, which is satisfied with not just a low loss, instead it seeks a neighborhood with uniformly low loss. SAM is motivated by the link between the geometry of the loss landscape and generalization. It makes sense that a low loss within a uniformly low loss neighborhood will generalize better than a low loss within a region of higher variance.

To be specific, we consider a model with the weight vector of $\mathbf{w}$ and the training loss $L_S$. SAM aims to minimize the maximum loss within a small region which is usually a $\ell_2$ ball with $\rho$ radius. Note that $\rho$ is a small value close to $0$. Therefore, SAM can be formulated as a minimax optimization problem:
$$\min_{\mathbf{w}} \max_{\mathbf{\epsilon}: \|\mathbf{\epsilon}\|_2\leq \rho} L_S (\mathbf{w} + \mathbf{\epsilon})$$

**(3)** (3 points) Please solve the inner maximum problem by first-order Taylor expansion.

<font color=‘blue’>

SAM approximates the inner maximization problem using a first-order Taylor expansion. The Taylor expansion of the loss function $L_S(\mathbf{w} + \mathbf{\epsilon})$ around $\mathbf{w}$ is given by:

$$L_S(\mathbf{w} + \mathbf{\epsilon}) \approx L_S(\mathbf{w}) + \mathbf{\epsilon}^T \nabla L_S(\mathbf{w})$$

To maximize this expression with respect to $\mathbf{\epsilon}$ under the constraint $\|\mathbf{\epsilon}\|_2 \leq \rho$, we set $\mathbf{\epsilon}$ in the direction of the gradient $\nabla L_S(\mathbf{w})$ with magnitude $\rho$:

$$\mathbf{\epsilon} = \rho \frac{\nabla L_S(\mathbf{w})}{\|\nabla L_S(\mathbf{w})\|_2}$$

**(4)** (8 points) Now we will train the same model using the SAM optimizer.
Please implement SAM by the two steps below. The first step is for the maximizer which calculates $\epsilon$ obtained in question (1). The second step is the normal step for the minimizer: $\mathbf{w}_{t+1} = \mathbf{w}_{t} - \eta_t \nabla L_S (\mathbf{w}_t + \mathbf{\epsilon}_t)$ where $\eta_t$ is step size. Note that we set $\rho=0.05$.

Hint: be careful about weight updates.

In [18]:
class SAM(torch.optim.Optimizer):
    def __init__(self, params, base_optimizer, lr=0.01, rho=0.05):
        assert rho >= 0.0, f"Invalid rho, should be non-negative: {rho}"

        defaults = dict(rho=rho)
        super(SAM, self).__init__(params, defaults)

        self.base_optimizer = base_optimizer(self.param_groups, lr)
        self.param_groups = self.base_optimizer.param_groups
        self.defaults.update(self.base_optimizer.defaults)

    def _grad_norm(self):
        # Note that p.grad gets the gradient; p.data gets the weight.
        norm = torch.norm(
                    torch.stack([
                        p.grad.norm(p=2)
                        for group in self.param_groups for p in group["params"]
                        if p.grad is not None
                    ]),
                    p=2
               )
        norm += 1e-12 # Avoid zero norm
        return norm

    @torch.no_grad()
    def first_step(self):
        # Add the perturbation on the weight.
        # Hint: the norm of the gradient can be calculated by _grad_norm() function.
        # Hint: self.param_groups to get access to the weight and gradient of each parameter
        # FILL
        grad_norm = self._grad_norm()

        for group in self.param_groups:
            rho = group["rho"]
            for p in group["params"]:
                if p.grad is None:
                    continue
                e_w = p.grad * (rho / grad_norm)
                p.add_(e_w)

        self.zero_grad()

    @torch.no_grad()
    def second_step(self, zero_grad=False):
        # FILL
        # Hint: Remember to change the parameters back
        self.base_optimizer.step()

        grad_norm = self._grad_norm()
        for group in self.param_groups:
            rho = group["rho"]
            for p in group["params"]:
                if p.grad is None:
                    continue
                e_w = p.grad * (rho / grad_norm)
                p.sub_(e_w)
        if zero_grad:
            self.zero_grad()

Define an optimizer of `SAM` for the model. We recommend using `SGD` as base optimizer with a learning rate of $0.05$ (which is same with SGD).

In [19]:
base_optimizer = torch.optim.SGD
sam_optimizer = SAM(sparse_model_sam.parameters(), base_optimizer, lr=0.05)

**(5)** (4 points) Please define the optimizing process of SAM.

In [20]:
def optimize_sam(model, optimizer, img, label):

    enable_running_stats(model)
    # First forward-backward pass
    # FILL
    # Hint: use sam_optimizer above
    # Hint: use loss 'cross_entropy'
    logits = model(img)                   
    loss = cross_entropy(logits, label) 
    loss.backward()                       
    optimizer.first_step()

    disable_running_stats(model)
    # Second forward-backward pass
    # FILL
    logits = model(img)                   
    loss = cross_entropy(logits, label) 
    loss.backward()                      
    optimizer.second_step() 

In [21]:
train_model(sparse_model_sam, sam_optimizer, optimize_sam, finetune_epoch)

Epoch 0 with 0.961 accuracy on the validation set.
Epoch 1 with 0.963 accuracy on the validation set.
Epoch 2 with 0.966 accuracy on the validation set.


#### Evaluate model
Evaluate the sparse model finetuned with SAM on the test set.

In [22]:
acc = evaluate(sparse_model_sam)
print(f"Accuracy of {round(acc, 4)} on the test set.")

Accuracy of 0.9779 on the test set.


**(6)** (2 points) Give a conclusion comparing SAM with SGD. Is there any drawback of SAM?

<font color=‘blue’>
    
**SGD**:
- Focuses on minimizing the loss function $L_S(\textbf{w})$ directly.
- It is sensitive to sharp minima.

**SAM**:
- Seeks to minimize the worst-case loss within a small neighborhood around the current weights, effectively minimizing: 
$$\min_{\textbf{w}} \max_{\mathbf{\epsilon}: \|\mathbf{\epsilon}\|_2\leq \rho} L_S (\mathbf{w} + \mathbf{\epsilon})$$
- Regularizes the optimization process by avoiding sharp minima, leading to flatter minima that often generalize better to unseen data.

**Drawbacks of SAM**:
- Increase of computational cost: requires two forward and backward passes for each optimization step.
- Sensitivity to hyperparameters: the radius $\rho$ and the learning rate $\eta$ need to be tuned carefully.